## Imports

In [ ]:
import requests
#from requests.structures import CaseInsensitiveDict
import json
import datetime
import pandas as pd
from requests.structures import CaseInsensitiveDict
from pandas.io.json import json_normalize
from datetime import date, timedelta
from datetime import date, timedelta
import sqlite3 as sql
import random
from datetime import datetime
from datetime import date

## Task 1.1 
<!--  Create function for : 
a. makes calls to fixer’s API (https://fixer.io/documentation)
b. fills a local SQlite database with currency information from 01.09.2018 to 30.09.2018. (Please use the default base currency, which is Euros) -->

In [2]:
### Function for filling local SQlite database with currency information from 01.09.2018 to 30.09.2018.(base EUR)

In [ ]:
json_resp=  []
def fetch_fixer(start_date,end_date):
    delta = timedelta(days=1)
    while start_date <= end_date:       # for fetching info for 01.09.2018 to 30.09.2018
        temp_date = start_date.strftime("%Y-%m-%d")
        url = "http://data.fixer.io/api/"+temp_date+"?access_key=&base=EUR"
        
        headers = CaseInsensitiveDict()
        headers["Accept"] = "application/json"
        resp = requests.get(url, headers=headers)
        json_resp.append(resp.json())
        df = json_normalize(json_resp)   ## in a normalize way where indexing is based on the dates
        resultdf = df.drop_duplicates()
        start_date += delta
    #print(resultdf)
    conn = sql.connect(r'\fixer.db')    # connecting to a db
    resultdf.to_sql('Rates', conn,if_exists='replace', index=False) # table creation 

In [ ]:
fetch_fixer(date(2018, 9, 1),date(2018, 9, 30))  

In [ ]:
#query execution
conn = sql.connect(r'\fixer.db')
pd.read_sql('SELECT * FROM Rates', conn)

## Task 1.2
<!-- function which
a. picks 5 random dates within the time period of 01.09.2018 and 30.09.2018.
b. acquires the data via API calls and checks if the values are accurate
c. throws an exception if they are not accurate -->

In [ ]:
## function that generates 5 random dates 

In [ ]:
def random_date(start, end,dates_temp):
    n = 5
    for i in range(n):
        random_date = start + (end - start) * random.random()
        dates_temp.append(random_date.strftime("%Y-%m-%d"))

In [3]:
## Retrieving values for these 5 dates from the fixer API and then storing it in a db

In [ ]:
json_resp_temp=  []
def accuracy_check(start_date,end_date):
    dates_temp=[]
    random_date(start_date, end_date,dates_temp)
    for date in dates_temp:
        print(date)
        url = "http://data.fixer.io/api/"+date+"?access_key=&base=EUR"
        headers = CaseInsensitiveDict()
        headers["Accept"] = "application/json"
        resp = requests.get(url, headers=headers)
        json_resp_temp.append(resp.json())
        df_temp = json_normalize(json_resp_temp)
        resultdf_temp = df_temp.drop_duplicates()
    conn = sql.connect(r'\fixer.db')
    resultdf_temp.to_sql('Rates_temp', conn, if_exists='replace', index=False)
    compare_date()

In [ ]:
##

In [ ]:
def compare_date():
    conn = sql.connect(r'\fixer.db')
    diff_rates=pd.read_sql('SELECT * FROM Rates_temp EXCEPT SELECT * FROM Rates', conn)
    n=len(diff_rates)
    if (n==0):
        print ("Values of respected dates are accurate")
    else:
        print ("There are some errors found for the specified dates")
        print (diff_rates['date'])
        raise Exception("Found Errors!!")

In [ ]:
accuracy_check(date(2018, 9, 1),date(2018, 9, 30))